# Data proccess


---
Here we try to extract more information frome the other files.

In [1]:
import numpy as np # linear algebra
import dask.dataframe as dd # data frame for larg datasets
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from scipy import stats
import gc
import time
import datetime

from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.layers import Dense, Conv2D, Flatten, Input,Dropout,BatchNormalization, Embedding, concatenate
from keras.regularizers import l2
from keras.utils import to_categorical
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

Using TensorFlow backend.


In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:

df_train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/train.csv',  parse_dates=['first_active_month'])
df_test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/test.csv',  parse_dates=['first_active_month'])



test have one na in date (need to deleate later)

In [0]:
df_train.head()

In [0]:
df_train.describe()

extract elapsed time feature

In [0]:
max_date = df_train['first_active_month'].dt.date.max()
def process_main(df):         
    df['elapsed_time'] = (max_date - df['first_active_month'].dt.date).dt.days
    return df

In [0]:
train = process_main(df_train)
test = process_main(df_test)

In [0]:
new_merchant_transactions = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/new_merchant_transactions.csv')


def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df


new_transactions = binarize(new_merchant_transactions)
new_transactions['purchase_date'] = pd.to_datetime(new_transactions['purchase_date'])

function to reduce the memory use of pandas data frame

In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# handelling with new_marchent_transactions

---



In [0]:
def aggregate_new_marchent_transactions(trans, prefix):
    trans['purchase_month'] = trans['purchase_date'].dt.month
    trans['month_diff'] = ((datetime.datetime.today() - trans['purchase_date']).dt.days)//30
    trans['month_diff'] += trans['month_lag']
    trans.loc[:, 'purchase_date'] = pd.DatetimeIndex(trans['purchase_date']).astype(np.int64) * 1e-9
    trans['installments'] = trans['installments'].astype(int)
    
    agg_func = {
        
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'max', 'min'],
        'month_lag': ['min', 'max']
        
    }
    agg_trans = trans.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (trans.groupby('card_id')
          .size()
          .reset_index(name='{}transactions_count'.format(prefix)))

    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')

    return agg_trans
  

In [16]:
%%time
gc.collect()
new_transactions = reduce_mem_usage(new_transactions)
history = aggregate_historical_transactions(new_transactions, prefix='new_')
del new_merchant_transactions
gc.collect()
train = pd.merge(train, history, on='card_id', how='left')
test = pd.merge(test, history, on='card_id', how='left')
del history
gc.collect()

Mem. usage decreased to 87.99 Mb (58.0% reduction)
CPU times: user 44.9 s, sys: 62 ms, total: 44.9 s
Wall time: 44.9 s


In [93]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
feature_1,201917.0,3.105311e+00,1.186160e+00,1.000000,2.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00
feature_2,201917.0,1.745410e+00,7.513617e-01,1.000000,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00
feature_3,201917.0,5.655690e-01,4.956833e-01,0.000000,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
target,201917.0,-3.936363e-01,3.850500e+00,-33.219281,-8.831102e-01,-2.343689e-02,7.654530e-01,1.796507e+01
elapsed_time,201917.0,3.819790e+02,2.937102e+02,0.000000,1.530000e+02,3.060000e+02,4.880000e+02,2.284000e+03
new_transactions_count,201917.0,6.040527e+00,6.663002e+00,0.000000,2.000000e+00,4.000000e+00,8.000000e+00,1.090000e+02
new_purchase_amount_sum,201917.0,NaN,0.000000e+00,-70.812500,-4.828125e+00,-2.156250e+00,-7.065430e-01,1.091250e+02
new_purchase_amount_mean,201917.0,NaN,0.000000e+00,-0.747070,-6.875000e-01,-6.240234e-01,-4.323730e-01,4.418750e+01
new_purchase_amount_max,201917.0,-0.000000e+00,0.000000e+00,-0.747070,-6.342773e-01,-4.614258e-01,0.000000e+00,7.668750e+01
new_purchase_amount_min,201917.0,NaN,0.000000e+00,-0.747070,-7.348633e-01,-7.182617e-01,-6.572266e-01,4.418750e+01


Clean the merged data for nan vals


In [0]:
for col in train.columns:
    if train[col].isna().any():
        train[col] = train[col].fillna(0)
    
for col in test.columns:
    if test[col].isna().any():
        test[col] = test[col].fillna(0)

In [0]:
train.to_csv('train_merge_new&H_transactions.csv')
test.to_csv('test_merge_new&H_transactions.csv')

In [0]:
filterd = train.query()

# handelling with history_transactions

In [0]:
history_transactions = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/historical_transactions.csv')


def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df




In [0]:
history_transactions = binarize(history_transactions)
history_transactions['purchase_date'] = pd.to_datetime(history_transactions['purchase_date'])

In [0]:
def aggregate_historical_transactions(trans, prefix):
    trans['purchase_month'] = trans['purchase_date'].dt.month
    trans['installments'] = trans['installments'].astype(int)
    trans['month_diff'] = ((datetime.datetime.today() - trans['purchase_date']).dt.days)//30
    trans['month_diff'] += trans['month_lag']
    trans.loc[:, 'purchase_date'] = pd.DatetimeIndex(trans['purchase_date']). \
                                        astype(np.int64) * 1e-9
    
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'max', 'min'],
        'month_lag': ['min', 'max']
    }
    agg_trans = trans.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (trans.groupby('card_id')
          .size()
          .reset_index(name='{}transactions_count'.format(prefix)))

    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')

    return agg_trans


In [11]:
%%time

gc.collect()
history_transactions = reduce_mem_usage(history_transactions)
history = aggregate_historical_transactions(history_transactions, prefix='hist_')
gc.collect()
train = pd.merge(train, history, on='card_id', how='left')
test = pd.merge(test, history, on='card_id', how='left')
del history

Mem. usage decreased to 1360.42 Mb (56.2% reduction)
CPU times: user 4min 10s, sys: 2.05 s, total: 4min 12s
Wall time: 4min 12s


In [0]:
train.to_csv('train_merge_history_transactions.csv')
test.to_csv('test_merge_history_transactions.csv')

# creating V4 data 

---
merging hist & new transactions and adding categorial features as one hot vec

In [0]:
hist = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/historical_transactions.csv')

In [0]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['start_year'] = df['first_active_month'].dt.year
    df['start_month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days



one hoting cat features

In [0]:
hist['authorized_flag'] = hist['authorized_flag'].map({'Y':1, 'N':0})
hist['category_1'] = hist['category_1'].map({'Y':1, 'N':0})

hist['category_2_1'] = (hist['category_2'] == 1) + 0
hist['category_2_2'] = (hist['category_2'] == 2) + 0
hist['category_2_3'] = (hist['category_2'] == 3) + 0
hist['category_2_4'] = (hist['category_2'] == 4) + 0
hist['category_2_5'] = (hist['category_2'] == 5) + 0

hist['category_3_A'] = (hist['category_3'].astype(str) == 'A') + 0
hist['category_3_B'] = (hist['category_3'].astype(str) == 'B') + 0
hist['category_3_C'] = (hist['category_3'].astype(str) == 'C') + 0

In [0]:
def aggregate_historical_transactions(history):
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['sum', 'mean'],
        'category_2': ['nunique'],
        'category_3_A': ['sum'],
        'category_3_B': ['sum'],
        'category_3_C': ['sum'],
        'category_2_1': ['sum','mean'],
        'category_2_2': ['sum','mean'],
        'category_2_3': ['sum','mean'],
        'category_2_4': ['sum','mean'],
        'category_2_5': ['sum','mean'],        
        'city_id': ['nunique'],
        'installments': ['sum', 'median', 'max', 'min', 'std'],
        'merchant_category_id': ['nunique'],
        'merchant_id': ['nunique'],
        'month_lag': ['min', 'max'],
        'purchase_amount': ['sum', 'median', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'max', 'min'],
        'state_id': ['nunique'],
        'subsector_id': ['nunique'],

        }
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['hist_' + '_'.join(col).strip() 
                           for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='hist_transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history



In [10]:
hist = reduce_mem_usage(hist)
history = aggregate_historical_transactions(hist)


Mem. usage decreased to 1582.53 Mb (67.6% reduction)


In [11]:
del hist
gc.collect()

446

In [0]:
new_merchant = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/project_2/new_merchant_transactions.csv")

In [0]:
new_merchant['authorized_flag'] = new_merchant['authorized_flag'].map({'Y':1, 'N':0})

new_merchant['category_1'] = new_merchant['category_1'].map({'Y':1, 'N':0})
new_merchant['category_3A'] = (new_merchant['category_3'].astype(str) == 'A') + 0
new_merchant['category_3B'] = (new_merchant['category_3'].astype(str) == 'B') + 0
new_merchant['category_3C'] = (new_merchant['category_3'].astype(str) == 'C') + 0

new_merchant['category_2x1'] = (new_merchant['category_2'] == 1) + 0
new_merchant['category_2x2'] = (new_merchant['category_2'] == 2) + 0
new_merchant['category_2x3'] = (new_merchant['category_2'] == 3) + 0
new_merchant['category_2x4'] = (new_merchant['category_2'] == 4) + 0
new_merchant['category_2x5'] = (new_merchant['category_2'] == 5) + 0
new_merchant['purchase_date'] = pd.DatetimeIndex(new_merchant['purchase_date']).\
                                      astype(np.int64) * 1e-9

In [0]:
def aggregate_new_transactions(new_trans):    
    
    
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['sum', 'mean'],
        'category_2': ['nunique'],
        'category_3A': ['sum'],
        'category_3B': ['sum'],
        'category_3C': ['sum'],     
        'category_2x1': ['sum','mean'],
        'category_2x2': ['sum','mean'],
        'category_2x3': ['sum','mean'],
        'category_2x4': ['sum','mean'],
        'category_2x5': ['sum','mean'],        

        'city_id': ['nunique'],
        'installments': ['sum', 'median', 'max', 'min', 'std'],
        'merchant_category_id': ['nunique'],
        'merchant_id': ['nunique'],
        'month_lag': ['min', 'max'],
        'purchase_amount': ['sum', 'median', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'max', 'min'],
        'state_id': ['nunique'],
        'subsector_id': ['nunique']        
        }
    agg_new_trans = new_trans.groupby(['card_id']).agg(agg_func)
    agg_new_trans.columns = ['new_' + '_'.join(col).strip() 
                           for col in agg_new_trans.columns.values]
    agg_new_trans.reset_index(inplace=True)
    
    df = (new_trans.groupby('card_id')
          .size()
          .reset_index(name='new_transactions_count'))
    
    agg_new_trans = pd.merge(df, agg_new_trans, on='card_id', how='left')
    
    return agg_new_trans



In [16]:
new_merchant = reduce_mem_usage(new_merchant)
new_trans = aggregate_new_transactions(new_merchant)

del new_merchant

Mem. usage decreased to 117.94 Mb (64.2% reduction)


In [17]:
print(df_train.shape)
print(df_test.shape)

xtrain = pd.merge(df_train, new_trans, on='card_id', how='left')
xtest = pd.merge(df_test, new_trans, on='card_id', how='left')

del new_trans

print(xtrain.shape)
print(xtest.shape)

xtrain = pd.merge(xtrain, history, on='card_id', how='left')
xtest = pd.merge(xtest, history, on='card_id', how='left')

del history

print(xtrain.shape)
print(xtest.shape)

(201917, 9)
(123623, 8)
(201917, 48)
(123623, 47)
(201917, 87)
(123623, 86)


In [18]:
xtrain.head(3)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,start_year,start_month,elapsed_time,new_transactions_count,...,hist_purchase_amount_sum,hist_purchase_amount_median,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_purchase_date_ptp,hist_purchase_date_max,hist_purchase_date_min,hist_state_id_nunique,hist_subsector_id_nunique
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,2017,6,245,23.0,...,-165.968735,-0.698042,2.258394,-0.739395,0.212139,20977987.0,1.519551e+09,1.498573e+09,3,21
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,2017,1,396,6.0,...,-210.006332,-0.708590,4.630300,-0.742400,0.384967,33717687.0,1.517438e+09,1.483720e+09,3,24
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,2016,8,549,1.0,...,-29.167391,-0.698868,-0.145847,-0.730138,0.087380,35635623.0,1.519759e+09,1.484123e+09,2,7


In [19]:

categorical_feats = ['feature_1', 'feature_2', 'feature_3']

for col in categorical_feats:
    print(col)
    lbl = LabelEncoder()
    lbl.fit(list(xtrain[col].values.astype('str')) + list(xtest[col].values.astype('str')))
    xtrain[col] = lbl.transform(list(xtrain[col].values.astype('str')))
    xtest[col] = lbl.transform(list(xtest[col].values.astype('str')))

feature_1
feature_2
feature_3


In [0]:
df_all = pd.concat([xtrain, xtest])
df_all = pd.get_dummies(df_all, columns=categorical_feats)

len_train = xtrain.shape[0]

xtrain = df_all[:len_train]
xtest = df_all[len_train:]

In [0]:
xtrain.to_csv('V4_train_new+hist+main.csv')
xtest.to_csv('V4_test_new+hist+main.csv')